In [1]:
import numpy as np
import os
import time
from vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
#from sklearn.utils import shuffle
#from sklearn.cross_validation import train_test_split

Using TensorFlow backend.


In [3]:
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

model = VGG16(include_top=True,input_tensor=image_input,weights=None)
#model = VGG16("qfeDf")
#model.summary()

In [5]:
last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(13, activation='softmax', name='output')(x)
vgg_model = Model(image_input, out)
#custom_vgg_model2.summary()

In [6]:
vgg_model.load_weights("weights/CheXNet_Keras_0.3.0_weights.h5")

ValueError: You are trying to load a weight file containing 242 layers into a model with 16 layers.

In [4]:
for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False

#custom_vgg_model2.summary()

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
data_path = 'sample_labels.csv'
NIH_Data = pd.read_csv(data_path)

In [7]:
NIH_Data["Finding Labels"][0].split('|')[0] == 'Emphysema'

True

In [8]:
train_data = NIH_Data[:2803] #50% of total data
test_data = NIH_Data[2803:5046] #40% of total data
validation_data = NIH_Data[5046:5607] #10% of total data

In [9]:
def getTarget(data):
    target = np.zeros([13])
    split_data = data.split('|')
    for i in range(0,len(split_data)):
        if(split_data[i] == 'Cardiomegaly'):
            target[0]=1
        if(split_data[i] == 'Emphysema'):
            target[1]=1
        if(split_data[i] == 'Atelectasis'):
            target[2]=1
        if(split_data[i] == 'Infiltration'):
            target[3]=1
        if(split_data[i] == 'Effusion'):
            target[4]=1
        if(split_data[i] == 'Pneumothorax'):
            target[5]=1
        if(split_data[i] == 'Pleural_Thickening'):
            target[6]=1
        if(split_data[i] == 'Edema'):
            target[7]=1
        if(split_data[i] == 'Mass'):
            target[8]=1
        if(split_data[i] == 'Nodule'):
            target[9]=1
        if(split_data[i] == 'Consolidation'):
            target[10]=1
        if(split_data[i] == 'Fibrosis'):
            target[11]=1
        if(split_data[i] == 'No Finding'):
            target[12]=1
            
    return target

In [10]:
def getDataTarget(data,start_val):
    data_targets = np.zeros([data.shape[0],13])
    for i in range(0,data.shape[0]):
        data_targets[i,:] = getTarget(data["Finding Labels"][i+start_val])
    return data_targets

In [11]:
def load_data(data,start_val):
    #print(data.shape[0])
    data_files = ["" for x in range(data.shape[0])]
    for i in range(0,data.shape[0]):
        #print(data["Image Index"][i+start_val])
        data_files[i] = ('images/'+data["Image Index"][i+start_val])
    data_files = np.asarray(data_files)
    #print(data_files)
    data_targets = getDataTarget(data,start_val)
    return data_files, data_targets

In [12]:
train_files, train_targets = load_data(train_data,0)
valid_files, valid_targets = load_data(validation_data,5046)
test_files, test_targets = load_data(test_data,2803)

In [13]:
NIH_Data[:1]

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139


In [14]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [15]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 2243/2243 [00:32<00:00, 68.40it/s]


In [16]:
custom_vgg_model2.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [17]:
hist = custom_vgg_model2.fit(train_tensors, train_targets, batch_size=32, 
                             epochs=12, verbose=1, validation_data=(valid_tensors, valid_targets))

Train on 2803 samples, validate on 560 samples
Epoch 1/12
2803/2803 [==============================] - 64s - loss: 2.4184 - acc: 0.5344 - val_loss: 2.6133 - val_acc: 0.4946
Epoch 2/12
2803/2803 [==============================] - 57s - loss: 2.2015 - acc: 0.5573 - val_loss: 3.0602 - val_acc: 0.4946
Epoch 3/12
2803/2803 [==============================] - 58s - loss: 2.1419 - acc: 0.5530 - val_loss: 2.5606 - val_acc: 0.4964
Epoch 4/12
2803/2803 [==============================] - 58s - loss: 2.0905 - acc: 0.5590 - val_loss: 2.5133 - val_acc: 0.4875
Epoch 5/12
2803/2803 [==============================] - 58s - loss: 2.0475 - acc: 0.5633 - val_loss: 2.5369 - val_acc: 0.4821
Epoch 6/12
2803/2803 [==============================] - 58s - loss: 1.9964 - acc: 0.5665 - val_loss: 2.5701 - val_acc: 0.4839
Epoch 7/12
2803/2803 [==============================] - 58s - loss: 1.9428 - acc: 0.5769 - val_loss: 2.6680 - val_acc: 0.4911
Epoch 8/12
2803/2803 [==============================] - 58s - loss: 1.9

In [18]:
(loss, accuracy) = custom_vgg_model2.evaluate(test_tensors, test_targets, batch_size=10, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

2243/2243 [==============================] - 52s    
[INFO] loss=2.4697, accuracy: 52.4744%


In [2]:
from keras.applications import resnet50
model1 = resnet50.ResNet50(include_top=True, weights='imagenet')
model1.load_weights('weights/CheXNet_Keras_0.3.0_weights.h5')

TypeError: softmax() got an unexpected keyword argument 'axis'

In [1]:
from keras.applications.densenet import DenseNet121
model1 = DenseNet121(include_top=True, weights='imagenet', classes=1000)
model1.load_weights('weights/CheXNet_Keras_0.3.0_weights.h5')
#model1 = keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)





Using TensorFlow backend.


TypeError: softmax() got an unexpected keyword argument 'axis'